# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [90]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import re
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score,recall_score,classification_report,f1_score,confusion_matrix,accuracy_score
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
import pickle

In [57]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [58]:
# load data from database
engine = create_engine('sqlite:///DisasterManagement.db')
df = pd.read_sql('select * from disaster_messages',engine)

In [59]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [61]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize (text)
    stemmed = [PorterStemmer().stem(w) for w in words]
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [62]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer = tokenize)),
                     ('tfidf',TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [63]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [66]:
X_train.head()

12160    here flood has affected 100.are we not valid f...
21725    Recent rains have slowly been recharging groun...
25498    Four new clinically compatible cases were repo...
22569    Pre- germinated seeds of photo insensitive see...
24732    The former French colony holds a third of the ...
Name: message, dtype: object

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [67]:
y_pred=pipeline.predict(X_test)

In [68]:
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i],average='macro')
        recall = recall_score(ArrayL[:, i], ArrayP[:, i],average='macro')
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i],average='macro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics

In [69]:
col_names = list(y.columns.values)
print(eval_metrics(np.array(y_test), y_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.799962   0.669656  0.553758  0.592953
request                 0.885393   0.859844  0.695882  0.741081
offer                   0.996949   0.498474  0.500000  0.499236
aid_related             0.730359   0.742439  0.706777  0.709217
medical_help            0.921243   0.799068  0.530267  0.537209
medical_products        0.951754   0.814696  0.545787  0.570088
search_and_rescue       0.968917   0.734727  0.508907  0.509853
security                0.981503   0.491032  0.499709  0.495332
military                0.966056   0.851866  0.536939  0.559305
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.943173   0.895645  0.617278  0.670775
food                    0.917429   0.877968  0.656408  0.709359
shelter                 0.916476   0.857380  0.595693  0.635574
clothing                0.986651   0.993314  0.556962  0.598907
money                   0.976354   0.838

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [70]:
parameters = {'vect__min_df': [1, 5],
              #'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 10)

# Find best parameters
#np.random.seed(74)
T_model = cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=1, score=0.21813760549277642, total= 1.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=1, score=0.23773422972393077, total= 1.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=1, score=0.22474964234620887, total= 1.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 12.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=1, score=0.22832618025751072, total= 1.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 14.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=5, score=0.24660277499642397, total= 1.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=5, score=0.24116721499070234, total= 1.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, vect__min_df=5, score=0.23977110157367668, total= 1.7min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, vect__min_df=1, score=0.21470462022600487, total=  54.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, vect__min_df=1, score=0.2131

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 40.3min finished


In [71]:
#grid search results
T_model.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([  51.62451577,   45.85255543,  103.63806287,   89.54330762,
          45.53410021,   43.47773043,   90.41756797,   83.47536008]),
 'std_fit_time': array([ 0.23046059,  0.48348356,  0.90415437,  0.31870985,  0.16457232,
         0.59105223,  0.1994971 ,  0.28611575]),
 'mean_score_time': array([  9.66080809,   9.34797581,  11.87843966,  11.12564977,
          9.68023403,   9.45788693,  12.00995469,  11.12151281]),
 'std_score_time': array([ 0.0807415 ,  0.04574278,  0.06496235,  0.07145078,  0.13007572,
         0.16734809,  0.05223594,  0.01894178]),
 'param_clf__estimator__min_samples_split': masked_array(data = [2 2 2 2 5 5 5 5],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_clf__estimator__n_estimators': masked_array(data = [10 10 25 25 10 10 25 25],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__min_df': masked_array(data = [1 5 1 5 1 5

In [72]:
# top mean score
np.max(T_model.cv_results_['mean_test_score'])

0.24251382796109097

In [73]:
# Parameters for top mean score
T_model.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 25,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [75]:
# evaluating metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.808352   0.676941  0.578346  0.599605
request                 0.901602   0.876024  0.748104  0.791588
offer                   0.996949   0.498474  0.500000  0.499236
aid_related             0.766400   0.766535  0.753651  0.757045
medical_help            0.923532   0.783518  0.566728  0.595047
medical_products        0.954615   0.868148  0.572203  0.611719
search_and_rescue       0.970633   0.819436  0.554329  0.587260
security                0.982075   0.491037  0.500000  0.495478
military                0.966247   0.829960  0.547337  0.575894
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.954996   0.934948  0.698564  0.766343
food                    0.943173   0.899800  0.790948  0.834108
shelter                 0.928108   0.881107  0.662968  0.718535
clothing                0.988177   0.946637  0.620060  0.687016
money                   0.975782   0.757

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
# summary of 1st model
Y_test_pred = pipeline.predict(X_test)
eval_metrics0 = eval_metrics(np.array(y_test), Y_test_pred, col_names)
eval_metrics0.describe()

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.940673,0.755011,0.578580,0.595211
std,0.060502,0.165700,0.110185,0.122058
min,0.730359,0.464095,0.499178,0.480998
25%,0.925391,0.662199,0.507170,0.499152
50%,0.957285,0.820781,0.529016,0.542006
75%,0.982742,0.859303,0.627060,0.676694
max,1.000000,1.000000,1.000000,1.000000


In [82]:
# summary of tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.946855,0.777740,0.610323,0.632511
std,0.053155,0.164745,0.127815,0.137726
min,0.766400,0.475472,0.499499,0.486469
25%,0.935545,0.723153,0.519302,0.520788
50%,0.959191,0.832656,0.553297,0.585215
75%,0.982790,0.880677,0.698751,0.744664
max,1.000000,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [91]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.